Библиотеки

In [21]:
from ast import While
from cmath import sqrt
from unittest.util import _MIN_DIFF_LEN
from sympy.tensor.array import derive_by_array
from scipy import optimize
import numpy as np
import sympy as sp
import math
import matplotlib.pyplot as plt
import scipy.optimize as opt
from sympy import *
import scipy


Константы

In [22]:
A = -0.024
B = 1.69
CC = 0.5
alpha = 10**-5 # K^-1 
T0 = 300 # K 
T_avg = 300 # K 
Young = 1.75 * 10**11 # Па
sigma_f = 1.1 * 10**8 # Па 
eps_f = 0.000628571
l = 10 # Длина стержня
T_f = 4 # Конечный момент времени  
h_t = 0.005 # Шаг времени 
a = 50 # Просто константа
n = 11 # Число узлов сетки
h = l/(n-1)  # Шаг сетки

Дифференцирование

Необходимые функции

In [23]:
def TT(x,t):
    return T_avg + F(x)*tau(t)

def F(x):
    return a*math.sin(pi*x/l)

# def tau(t):
#     return t*math.sin(t)

def tau(t):
    return (t+1)*math.cos(pi*(t**2))

def EpsT(x,t):
   alpha * (TT(x,t) - T0)


In [24]:
from sympy import diff, symbols, cos, sin
x = sp.Symbol('x')
t = sp.Symbol('t')
duu = alpha * (T_avg + (a * sin(pi * x/l)) * (t * sin(t)) - T0) 
u = duu.diff(x)
print(u)


5.0e-5*pi*t*sin(t)*cos(pi*x/10)


In [25]:
def isCorrectArray(a):
    n = len(a)
    
    for row in range(0, n):
        if( len(a[row]) != n ):
            print('Не соответствует размерность')
            return False
        
    for row in range(1, n - 1):
        if(abs(a[row][row]) < abs(a[row][row - 1]) + abs(a[row][row + 1])):
            print('Не выполнены условия достаточности')
            return False
 
    if (abs(a[0][0]) < abs(a[0][1]))or(abs(a[n - 1][n - 1]) < abs(a[n - 1][n - 2])):
        print('Не выполнены условия достаточности')
        return False
        
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
 
 
# Процедура нахождения решения 3-х диагональной матрицы
def solution(a, b):
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
        return -1 
 
    n = len(a)
    x = [0 for k in range(0, n)] # обнуление вектора решений
    print('Размерность матрицы: ',n,'x',n)
    
    # Прямой ход
    v = [0 for k in range(0, n)]
    u = [0 for k in range(0, n)]
    # для первой 0-й строки
    v[0] = a[0][1] / (-a[0][0]) 
    u[0] = ( - b[0]) / (-a[0][0]) 
    for i in range(1, n - 1): # заполняем за исключением 1-й и (n-1)-й строк матрицы
        v[i] = a[i][i+1] / ( -a[i][i] - a[i][i-1]*v[i-1] )
        u[i] = ( a[i][i-1]*u[i-1] - b[i] ) / ( -a[i][i] - a[i][i-1]*v[i-1] )
    # для последней (n-1)-й строки
    v[n-1] = 0
    u[n-1] = (a[n-1][n-2]*u[n-2] - b[n-1]) / (-a[n-1][n-1] - a[n-1][n-2]*v[n-2])
    
    print_arr('Прогоночные коэффициенты v: ','v', v)
    print_arr('Прогоночные коэффициенты u: ','u', u)
    
    # Обратный ход
    x[n-1] = u[n-1]
    for i in range(n-1, 0, -1):
        x[i-1] = v[i-1] * x[i] + u[i-1]
        
    return x    

Численное решение (Метод конечных разностей)

In [27]:
sigma = np.zeros(n)
sigma_fv = np.full(n,sigma_f)
eps = np.zeros(n)
eps_e = np.zeros(n)
eps_T = np.zeros(n)
eps_crk = np.zeros(n)
T = np.zeros(n)


points = [i for i in range(n+1)]
# H = np.diag(1/(h**2),-1)+ np.diag(2/(h**2), 0) + np.diag(1/h**2,1)
H = np.diag(np.full(n+1,-2))+np.diag(np.ones(n),1)+np.diag(np.ones(n),-1)
H[0,0]=1
H[0,1]=0
H[n,n]=1
H[n,n-1]=0
print(H)

def f1(x,t):
    return a*alpha*pi*(1+t) * cos(pi*(t**2))*cos(pi*x/l)

values = [0.0]
for i in range(1,n-1):
    values.append(f1(i,t))
    # print(i)
values.append(0)
print(len(values))
# print(values)

# u = np.dot(np.linalg.inv(H),values)
# print(u)
u = scipy.linalg.solve(H, values)
print(len(u))
du = np.array([0])
for i in range(1,n):
    du = np.append(du,(u[i+1]-u[i-1])/(2*h)) #центральная производная

print(len(du))

# def fff(q,tt):
#     t = tt
#     return q

# data = np.array([])

# T_array = np.arange(0,T_f+h_t,h_t)
# for tt in T_array:
#     temp = np.array([])
#     for i in range(0,n-1):
#         T[i] = TT(points[i],tt)
#         eps[i] = float(du[i]) #полные деформации
#         print(eps[i])
#         eps_T[i] = alpha * (T[i] - T0) #температурные деформации
#         eps_e[i] = eps[i]-eps_T[i]-eps_crk[i] #упругие деформации
#         if Young * eps_e[i] < sigma_fv[i]:
#             sigma[i] = Young * eps_e[i]
#         else:
#             sigma_fv[i] = sigma_f * (A+B*exp(-CC * (eps[i] - eps_T[i])/eps_f))
#             sigma[i] = sigma_fv[i]
#             eps_crk[i] = eps[i] - eps_T[i] - sigma[i]/Young
#             def g(x):
#                 return sigma_f * (A+B*exp(-CC * (x + eps_crk[i])/eps_f))-sigma_fv[i]
#             # eps_e[i] = optimize.newton(g, 1.5)
#             # eps_e[i]= optimize.root(g, -400000)
#         # print(eps[i])
#         temp = np.append(temp,[tt,T[i],sigma[i],eps[i],eps[i]-eps_T[i],eps_crk[i]])
#     data = np.append(data,temp)

# data = data.reshape(int((T_f/h_t)+1),((n-1)-1+1),6)


[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]
11


ValueError: object arrays are not supported

***Графики***

In [ ]:
times = int(T_f/h_t)
# print(data[0][0])
time = [(lambda i: data[i][1][0])(i) for i in range(0, times)]
eps = [(lambda i: data[i][1][3])(i) for i in range(0,times)]
sigma = [(lambda i: data[i][1][2])(i) for i in range(0,times)]
eps_eps_T = [(lambda i: data[i][1][4])(i) for i in range(0,times)]
eps_crk = [(lambda i: data[i][1][5])(i) for i in range(0,times)]
norm_eps = [(lambda i: data[i][1][4]/eps_f)(i) for i in range(0,times)]
norm_sigma = [(lambda i: data[i][1][2]/sigma_f)(i) for i in range(0,times)]
T = [(lambda i: data[i][1][1])(i) for i in range(0,times)]

# print(eps)
fig = plt.figure(figsize=(12,12))

axes1 = fig.add_subplot(3, 2, 1)
axes2 = fig.add_subplot(3, 2, 2)
axes3 = fig.add_subplot(3, 2, 3)
axes4 = fig.add_subplot(3, 2, 4)
axes5 = fig.add_subplot(3, 2, 5)
axes6 = fig.add_subplot(3, 2, 6)


axes1.plot(time,eps)
axes1.set_xlabel('$t$')
axes1.set_ylabel('$eps$')

axes2.plot(time,sigma)
axes2.set_xlabel('$t$')
axes2.set_ylabel('$sigma$')

axes3.plot(time,eps_crk)
axes3.set_xlabel('$t$')
axes3.set_ylabel('$eps_{crk}$')

axes4.plot(eps_eps_T,sigma)
axes4.set_xlabel('$eps-eps_T$')
axes4.set_ylabel('$sigma$')

axes5.plot(norm_eps,norm_sigma)
axes5.set_xlabel('$norm_{eps}$')
axes5.set_ylabel('$norm_{sigma}$')

axes6.plot(time,T)
axes6.set_xlabel('$t$')
axes6.set_ylabel('$T$')

fig. tight_layout (h_pad=5)
# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

plt.show()

NameError: name 'data' is not defined